# Open Banking Directory explorer
Use this notebook to query the Open Banking directory for sandboxes

NB: You need provide your own copy of the latest ASPSPs.json

In [ ]:
from pathlib import Path
import json

aspsps_file = Path("./local/ASPSPs.json")
assert aspsps_file.is_file(), "You need to download the data (* Fintech > OBIE > Open Banking Get ASPSPs > Results)"

aspsps = None
with aspsps_file.open('r') as handle:
    data = handle.read()
    aspsps = json.loads(data)

resources = [resource for resource in aspsps['Resources']]

In [ ]:
print('List of ASPSPs that are active:')
count = 0
for resource in resources:
    name = resource['urn:openbanking:organisation:1.0']['OrganisationCommonName']
    ob_id = resource['urn:openbanking:organisation:1.0']['OBOrganisationId']
    has_auth = 'AuthorisationServers' in resource
    is_active = any(auth['Active'] for auth in resource["urn:openbanking:competentauthorityclaims:1.0"]['Authorisations'])
    
    if is_active:
        print(ob_id,'\t', name, '\t', has_auth)
        count += 1

print(count, ' banks')

In [ ]:
print('List of ASPSPs that are active and have an Authorisation Server defined:')
count = 0
for resource in resources:
    name = resource['urn:openbanking:organisation:1.0']['OrganisationCommonName']
    ob_id = resource['urn:openbanking:organisation:1.0']['OBOrganisationId']
    has_auth = 'AuthorisationServers' in resource
    is_active = any(auth['Active'] for auth in resource["urn:openbanking:competentauthorityclaims:1.0"]['Authorisations'])
    
    if is_active and has_auth:
        for auth in resource['AuthorisationServers']:
            friendly_name = auth.get('CustomerFriendlyName')
            portal_url = auth.get('DeveloperPortalUri')
            config_url = auth.get('OpenIDConfigEndPointUri')
            
            print(f"{ob_id}\t{friendly_name}\t{portal_url}\t{config_url}\t({name})")
            count += 1

print(count, ' Auth servers')

In [ ]:
import requests

print('List of ASPSPs that are active and have an Authorisation Server defined:')
count = 0
for resource in resources:
    name = resource['urn:openbanking:organisation:1.0']['OrganisationCommonName']
    ob_id = resource['urn:openbanking:organisation:1.0']['OBOrganisationId']
    has_auth = 'AuthorisationServers' in resource
    is_active = any(auth['Active'] for auth in resource["urn:openbanking:competentauthorityclaims:1.0"]['Authorisations'])
    
    if is_active and has_auth:
        for auth in resource['AuthorisationServers']:
            friendly_name = auth.get('CustomerFriendlyName')
            portal_url = auth.get('DeveloperPortalUri')
            config_url = auth.get('OpenIDConfigEndPointUri')
            
            auth_methods = []
            auth_url = None
            token_url = None
            register_url = None
            if config_url:
                try:
                    resp = requests.get(config_url, timeout=5)
                    resp.raise_for_status()
                    config = resp.json()
                    auth_methods = config.get('token_endpoint_auth_methods_supported', [])
                    auth_url = config.get('authorization_endpoint')
                    token_url = config.get('token_endpoint')
                    register_url = config.get('registration_endpoint')
                except Exception as ex: 
                    pass # print(ex)
            
            print(f"{ob_id}\t{friendly_name}\t{portal_url}\t{len(auth_methods)}\t{'Y' if register_url else 'N'}\t{'Y' if auth_url else 'N'}\t{'Y' if token_url else 'N'}\t{name}\t{config_url}")
            count += 1

print(count, ' Auth servers')